# Предсказание рейтинга ресторана по версии TripAdvisor

In [1]:
import pandas as pd
import re

### Предобработка данных

In [2]:
df = pd.read_csv('main_task.csv')

In [3]:
no_capitals=["Munich", "Oporto", "Milan", "Barcelona","Zurich", "Lyon", "Hamburg", "Geneva", "Krakow"]
def capital(city):
    if city in no_capitals:
        return 0
    return 1
df['Capital']=df['City'].apply(capital)

# Добавление столбца с количеством типов кухонь в ресторане
df['Cuisine Style']=df['Cuisine Style'].fillna('Unknown')
df['Cuisine Style'] = df['Cuisine Style'].apply(lambda x: x.replace('[','').replace(']','').replace("'",'').replace(' ','').lstrip().split(','))
df['Cuisine Style Number'] = df['Cuisine Style'].apply(lambda x: len(x))

# Добавление столбцов с ценовыми категориями ресторанов
df['Price Range']=df['Price Range'].fillna('Unknown')
def high_price(price):
    if "$$$$" in price:
        return 1
    else:
        return 0
df['High Price']=df['Price Range'].apply(high_price)
def middle_price(price):
    if "$$ - $$$" in price:
        return 1
    else:
        return 0
df['Middle Price']=df['Price Range'].apply(middle_price)
def low_price(price):
    if len(price)==1:
        return 1
    else:
        return 0
df['Low Price']=df['Price Range'].apply(low_price)
def unknown_price(price):
    if price=="Unknown":
        return 1
    else:
        return 0
df['Unknown Price']=df['Price Range'].apply(unknown_price)

# Добавление столбца с ресторанами, в меню которых представлено вегетарианская кухня
def vf(styles):
    if "Vegetarian Friendly" in styles:
        return 1
    else:
        return 0
df['Vegetarian Friendly']=df['Cuisine Style'].apply(vf)

# Добавление столбца с ресторанами, о которых хорошо отзываются
def good_reviews(review):
    if ('good' or 'excellent' or 'best' or 'nice' or 'great') in review:
        return 1
    else:
        return 0
df['Good reviews']=df['Reviews'].apply(good_reviews)

# Добавление столбца с разницей во времени между двумя отзывами
pattern = re.compile('\d\d\W\d\d\W\d\d\d\d')
def make_times_list(reviews):
    return pattern.findall(reviews) 
df['review_times']=df['Reviews'].apply(make_times_list)
def lr(review_times):
    if len(review_times)==2:
        return review_times[0]
    else:
        return 0
df['last_review']=df['review_times'].apply(lr)
def pr(review_times):
    if len(review_times)==2:
        return review_times[1]
    else:
        return 0
df['penultimate_review']=df['review_times'].apply(pr)
df['last_review'] = pd.to_datetime(df['last_review'])
df['penultimate_review'] = pd.to_datetime(df['penultimate_review'])
df['time_difference']=df['last_review']-df['penultimate_review']
df['time_difference']=df['time_difference'].dt.days
def days_number(days):
    if days<0:
        return (-days)
    elif days==0:
        return df['time_difference'].mean()
    else:
        return days
df['time_difference']=df['time_difference'].apply(days_number)
del df['review_times']
del df['last_review']
del df['penultimate_review']

# Dummy variables
# Кухни
styles = set()
for i in range(len(df['Cuisine Style'])):
    for style in df['Cuisine Style'][i]:
        styles.add(style)
def find_item(cell):
    if item in cell:
        return 1
    return 0
for item in styles:
    df[item] = df['Cuisine Style'].apply(find_item)
# Города
cities = set()
for i in range(len(df['City'])):
    cities.add(df['City'][i])
def find_city(cell):
    if city in cell:
        return 1
    return 0
for city in cities:
    df[city] = df['City'].apply(find_city)

# Удаление столбцов
df=df.drop(['City','Cuisine Style','Price Range','Reviews','URL_TA','ID_TA'], axis=1)
df=df.fillna(df.mean())

In [4]:
df1 = pd.read_csv('kaggle_task.csv')

# Добавление столбца-индикатора столиц
no_capitals=["Munich", "Oporto", "Milan", "Barcelona","Zurich", "Lyon", "Hamburg", "Geneva", "Krakow"]
def capital(city):
    if city in no_capitals:
        return 0
    return 1
df1['Capital']=df1['City'].apply(capital)

# Добавление столбца с количеством типов кухонь в ресторане
df1['Cuisine Style']=df1['Cuisine Style'].fillna('Unknown')
df1['Cuisine Style'] = df1['Cuisine Style'].apply(lambda x: x.replace('[','').replace(']','').replace("'",'').replace(' ','').lstrip().split(','))
df1['Cuisine Style Number'] = df1['Cuisine Style'].apply(lambda x: len(x))

# Добавление столбцов с ценовыми категориями ресторанов
df1['Price Range']=df1['Price Range'].fillna('Unknown')
def high_price(price):
    if "$$$$" in price:
        return 1
    else:
        return 0
df1['High Price']=df1['Price Range'].apply(high_price)
def middle_price(price):
    if "$$ - $$$" in price:
        return 1
    else:
        return 0
df1['Middle Price']=df1['Price Range'].apply(middle_price)
def low_price(price):
    if len(price)==1:
        return 1
    else:
        return 0
df1['Low Price']=df1['Price Range'].apply(low_price)
def unknown_price(price):
    if price=="Unknown":
        return 1
    else:
        return 0
df1['Unknown Price']=df1['Price Range'].apply(unknown_price)

# Добавление столбца с ресторанами, в меню которых представлено вегетарианская кухня
def vf(styles):
    if "Vegetarian Friendly" in styles:
        return 1
    else:
        return 0
df1['Vegetarian Friendly']=df1['Cuisine Style'].apply(vf)

# Добавление столбца с ресторанами, о которых хорошо отзываются
df1['Reviews']=df1['Reviews'].fillna('')
def good_reviews(review):
    if ('good' or 'excellent' or 'best' or 'nice' or 'great') in review:
        return 1
    else:
        return 0
df1['Good reviews']=df1['Reviews'].apply(good_reviews)

# Добавление столбца с разницей во времени между двумя отзывами
pattern = re.compile('\d\d\W\d\d\W\d\d\d\d')
def make_times_list(reviews):
    return pattern.findall(reviews) 
df1['review_times']=df1['Reviews'].apply(make_times_list)
def lr(review_times):
    if len(review_times)==2:
        return review_times[0]
    else:
        return 0
df1['last_review']=df1['review_times'].apply(lr)
def pr(review_times):
    if len(review_times)==2:
        return review_times[1]
    else:
        return 0
df1['penultimate_review']=df1['review_times'].apply(pr)
df1['last_review'] = pd.to_datetime(df1['last_review'])
df1['penultimate_review'] = pd.to_datetime(df1['penultimate_review'])
df1['time_difference']=df1['last_review']-df1['penultimate_review']
df1['time_difference']=df1['time_difference'].dt.days
def days_number(days):
    if days<0:
        return (-days)
    elif days==0:
        return df1['time_difference'].mean()
    else:
        return days
df1['time_difference']=df['time_difference'].apply(days_number)
del df1['review_times']
del df1['last_review']
del df1['penultimate_review']

# Dummy variables
# Кухни

def find_style(cell):
    if style in cell:
        return 1
    return 0
for style in styles:
    df1[style] = df1['Cuisine Style'].apply(find_style)
# Города
def find_city(cell):
    if city in cell:
        return 1
    return 0
for city in cities:
    df1[city] = df1['City'].apply(find_city)

# Удаление столбцов
df1=df1.drop(['Name','City','Cuisine Style','Price Range','Reviews','URL_TA','ID_TA'], axis=1)
df1=df1.fillna(df.mean())

### Обучение модели и предсказание значений целевой переменной

In [12]:
# Х - данные с информацией о ресторанах, у - целевая переменная (рейтинги ресторанов)
X = df.drop(['Restaurant_id', 'Rating'], axis = 1)
y = df['Rating']
# Загружаем специальный инструмент для разбивки:
from sklearn.model_selection import train_test_split
# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.
# Для тестирования мы будем использовать 25% от исходного датасета.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1)
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели
# Создаём модель
regr = RandomForestRegressor(n_estimators=100)

# Обучаем модель на тестовом наборе данных
regr.fit(X_train, y_train)

X1 = df1.drop(['Restaurant_id'], axis = 1)
y_pred = regr.predict(X1)
df_end=pd.DataFrame(df1['Restaurant_id'])
df_end['Rating']=pd.Series(y_pred)
df_end['Rating']=df_end['Rating'].apply(lambda x: round(x*2)/2)
df_end.head()

,Restaurant_id,Rating
0,id_0,3.0
1,id_1,4.5
2,id_2,4.5
3,id_3,4.5
4,id_4,4.5


In [13]:
df_end['Rating']=df_end['Rating'].apply(lambda x: round(x*2)/2)
df_end.head()

,Restaurant_id,Rating
0,id_0,3.0
1,id_1,4.5
2,id_2,4.5
3,id_3,4.5
4,id_4,4.5


In [14]:
df_end.to_csv('solution.csv', index = False)